## Job Crafting: Evaluating Robustness Check

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from scipy.stats import spearmanr
import random

In [ ]:
# import results saved from 07_checking-robustness-a.py
PATH = f'robustness_check/'

random_seeds = np.load(PATH + f'data/random_seeds.npy')
labels = np.load(PATH + f'data/labels.npy')
no_clusters = np.load(PATH + f'data/no_clusters.npy')
rand_indices = np.load(PATH + f'data/ARI.npy')

### Examine: Number of clusters

In [ ]:
# plot frequency of number of clusters over different UMAP runs
fig, ax = plt.subplots(1, 1, figsize=(15, 5))

ax.set_facecolor('ghostwhite')
ax.grid(axis='y', color='lightgrey')
ax.set_axisbelow(True)
ax.set_xlabel('Number of clusters', fontsize=14)
ax.set_ylabel('Frequency', fontsize=14)

for pos in ('top', 'right', 'bottom', 'left'):
    ax.spines[pos].set_visible(False)

ax.hist(no_clusters, bins=np.arange(-0.5, 51, 1), rwidth=0.9, color='cornflowerblue')

fig.savefig(PATH + 'no-clusters.svg')

In [ ]:
# We define trivial solutions as solutions outside the mass of the normal distribution,
# that is with number of clusters below 27.

# create new variables with trivial solutions deleted
random_seeds_trivial = np.delete(random_seeds, np.where(no_clusters < 27)[0])
no_clusters_trivial = np.delete(no_clusters, np.where(no_clusters < 27)[0])
rand_trivial = np.delete(rand_indices, np.where(no_clusters < 27)[0], 0)
rand_trivial = np.delete(rand_trivial, np.where(no_clusters < 27)[0], 1)

In [ ]:
# print parameters of cluster number distribution
print(f'Mean of cluster distribution without trivial solutions: \
{round(np.mean(no_clusters_trivial), 4)}')

print(f'SD of cluster distribution without trivial solutions: \
{round(np.std(no_clusters_trivial), 4)}\n')

print('Prevalence of numbers of found clusters:')
pd.Series(no_clusters).value_counts()

### Examine: Rand Indices

In [ ]:
# get min and max of rand indices for pairs of clusterings *with* trivial solutions
print(f'Min and max of rand indices with trivial solutions:')
print(f'Min: {round(np.min([x for x in rand_indices.flatten() if x != 1]), 4)}')
print(f'Max: {round(np.max([x for x in rand_indices.flatten() if x != 1]), 4)}')

In [ ]:
# get min and max of rand indices for pairs of clusterings *without* trivial solutions
print(f'Min and max of rand indices without trivial solutions:')
print(f'Min: {round(np.min([x for x in rand_trivial.flatten() if x != 1]), 4)}')
print(f'Max: {round(np.max([x for x in rand_trivial.flatten() if x != 1]), 4)}')

In [ ]:
# get distribution of average rand indices per random seed *without* trivial solutions
avg_rand = [(sum(rand_trivial[i])-1)/(len(rand_trivial)-1) for i in range(len(rand_trivial))]

fig, ax = plt.subplots(figsize=(12, 6))
plt.hist(avg_rand, bins=np.arange(0.5, 0.90, 0.01), color='cornflowerblue')

for pos in ('top', 'right', 'bottom', 'left'):
    ax.spines[pos].set_visible(False)

ax.set_facecolor('ghostwhite')
ax.grid(axis='y', color='lightgrey', linewidth=1)
ax.set_axisbelow(True)
ax.set_xlabel('Average Adjusted Rand Index', fontsize=14)
ax.set_ylabel('Number of solutions', fontsize=14)

plt.savefig(PATH + 'avg_ARI.svg')

In [ ]:
# calculate correlation between rand index and difference in number of clusters per cluster pairing

# calculate cluster differences between cluster pairings in 5000x5000 array
cluster_diff = []
for i in range(len(no_clusters)):
    cluster_diff.append(no_clusters[i] - no_clusters)
cluster_diff = np.array(cluster_diff)

# keep lower triangle of cluster differences array only, flatten, make values absolute
cluster_diff_tril = []
for i in range(1, len(cluster_diff)):
    cluster_diff_tril.append(cluster_diff[i, :i])
cluster_diff_flat = [abs(x) for xs in cluster_diff_tril for x in xs]

# keep lower triangle of rand indices array only, flatten
rand_indices_tril = []
for i in range(1, len(rand_indices)):
    rand_indices_tril.append(rand_indices[i, :i])
rand_indices_flat = [x for xs in rand_indices_tril for x in xs]

# correlate
print(f'Differences in numbers of clusters correlated with rand indices per cluster pairing yields: \
{round(spearmanr(rand_indices_flat, cluster_diff_flat).statistic, 4)} (Spearman)')

### Choose final UMAP random seed

In [ ]:
# randomly choose random seed that produces 40 clusters (majority of cluster solutions)
# random.choice(random_seeds[np.where(no_clusters == 40)])
print("Random seed yielding 40 clusters: 44669")

In [ ]:
print(f"Random majority solution has average ARI of {round(avg_rand[np.where(random_seeds_trivial == 44669)[0][0]], 4)}.")

In [ ]:
print(f"The mean of average Adjusted Rand Indices after excluding trivial solutions is {round(np.mean(avg_rand), 4)}, \
the minimum is {round(np.min(avg_rand), 4)}, \
and the maximum is {round(np.max(avg_rand), 4)}.")